In [0]:
from pyspark.sql.functions import col

**Get Secrets**

In [0]:
secrets_scope_name = "tokyo-olympics-secrets"

app_secret = dbutils.secrets.get(scope = secrets_scope_name, key = "app-secret")
app_id = dbutils.secrets.get(scope = secrets_scope_name, key = "app-id")
tenant_id = dbutils.secrets.get(scope = secrets_scope_name, key = "tenant-id")

In [0]:
data_lake_name = "tokyoolympicdatasonakshi"

In [0]:
spark.conf.set(f"fs.azure.account.auth.type.{data_lake_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{data_lake_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{data_lake_name}.dfs.core.windows.net", app_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{data_lake_name}.dfs.core.windows.net", app_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{data_lake_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
spark

In [0]:
athletes = spark.read.format("csv") \
    .option("header",True) \
    .option("inferSchema",True) \
    .load("abfss://tokyo-olympic-data@tokyoolympicdatasonakshi.dfs.core.windows.net/raw-data/Athletes.csv")

coaches = spark.read.format("csv") \
    .option("header",True) \
    .option("inferSchema",True) \
    .load("abfss://tokyo-olympic-data@tokyoolympicdatasonakshi.dfs.core.windows.net/raw-data/Coaches.csv")

entries_gender = spark.read.format("csv") \
    .option("header",True) \
    .option("inferSchema",True) \
    .load("abfss://tokyo-olympic-data@tokyoolympicdatasonakshi.dfs.core.windows.net/raw-data/EntriesGender.csv")

medals = spark.read.format("csv") \
    .option("header",True) \
    .option("inferSchema",True) \
    .load("abfss://tokyo-olympic-data@tokyoolympicdatasonakshi.dfs.core.windows.net/raw-data/Medals.csv")

teams = spark.read.format("csv") \
    .option("header",True) \
    .option("inferSchema",True) \
    .load("abfss://tokyo-olympic-data@tokyoolympicdatasonakshi.dfs.core.windows.net/raw-data/Teams.csv")

In [0]:
athletes.show(5)

In [0]:
athletes.printSchema()

In [0]:
coaches.show(5)

In [0]:
coaches.printSchema()

In [0]:
entries_gender.show(5)

In [0]:
entries_gender.printSchema()

In [0]:
medals.show(5)

In [0]:
medals.printSchema()

In [0]:
teams.show(5)

In [0]:
teams.printSchema()

In [0]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy(col("Gold"), ascending=False).select("TeamCountry","Gold").show()

In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entries_gender.withColumn(
    'Avg_Female', entries_gender['Female']/entries_gender['Total']
).withColumn(
    'Avg_Male', entries_gender['Male']/entries_gender['Total']
)

average_entries_by_gender.show()

In [0]:
athletes.repartition(1).write.mode("overwrite").option("header",True).csv("abfss://tokyo-olympic-data@tokyoolympicdatasonakshi.dfs.core.windows.net/transformed-data/athletes")

coaches.repartition(1).write.mode("overwrite").option("header",True).csv("abfss://tokyo-olympic-data@tokyoolympicdatasonakshi.dfs.core.windows.net/transformed-data/coaches")

entries_gender.repartition(1).write.mode("overwrite").option("header",True).csv("abfss://tokyo-olympic-data@tokyoolympicdatasonakshi.dfs.core.windows.net/transformed-data/entriesgender")

medals.repartition(1).write.mode("overwrite").option("header",True).csv("abfss://tokyo-olympic-data@tokyoolympicdatasonakshi.dfs.core.windows.net/transformed-data/medals")

teams.repartition(1).write.mode("overwrite").option("header",True).csv("abfss://tokyo-olympic-data@tokyoolympicdatasonakshi.dfs.core.windows.net/transformed-data/teams")